In [2]:
import numpy as np
import json
import time, os
import math

# Global variables / dataset 
panoptic_path = "../nas/MAEVE/dataset/panoptic-toolbox/"
sequences = ["170915_office1","160422_ultimatum1","160906_band4","161029_build1","161029_sports1","161029_tools1","170407_haggling_a1"]
sequence_name = "161029_sports1"

sequence_path = os.path.join(panoptic_path,sequence_name)

CONTINUOUS_STATE_PARTS = [
            "nose", "left_ear", "right_ear", "left_shoulder", "right_shoulder", 
            "left_elbow", "right_elbow", "left_wrist", "right_wrist", "left_hip", "right_hip", "left_knee", 
            "right_knee", "left_ankle", "right_ankle", "neck"]

mapping_panoptic_cstate = [1,16,18,3,9,4,10,5,11,6,12,7,13,8,14,0]

In [3]:
# Merge a unique json with ground truth
gt = []
directory= os.path.join(sequence_path,"hdPose3d_stage1_coco19")

for filename in sorted(os.listdir(directory)):
    with open(os.path.join(directory,filename),'r') as f:
        data  = json.load(f)
        json_frame = {}
        json_frame["frame_id"] = int(filename.split('_')[1].split('.')[0])
        json_frame["timestamp"] = data["univTime"]
        # print(json_frame["timestamp"])
        json_frame["kp3d"] = []
        json_frame["kp2d"] = []
        json_frame["continuousState"] = []
        json_frame["track_ids"] = []
        for body in data["bodies"]:
            json_frame["track_ids"].append(body["id"])
            kp19 = np.array(body["joints19"]).reshape(-1,4)
            kp18 = kp19[mapping_panoptic_cstate,0:3]/100
            acc = kp19[mapping_panoptic_cstate,3]*100 # Because it's too low for visualization
            json_frame["continuousState"].append(kp18.tolist())
            kp3d = {}
            kp2d = {}
            for i,label in enumerate(CONTINUOUS_STATE_PARTS):
                kp3d[label] = kp18[i,:].tolist()    
                kp2d[label] = [0,0,acc[i]]
            json_frame["kp3d"].append(kp3d)
            json_frame["kp2d"].append(kp2d)
        gt.append(json_frame)

# dump json
with open(os.path.join("tmp/",sequence_name+".gt.json"), "w") as outfile:
    json.dump(gt, outfile)

# Export in viewer format
os.system('python3 to_viewer.py tmp/'+sequence_name+'.gt.json --rotation 180 --output "tmp/"'+sequence_name+'".gt.viewer.json" >> "tmp/"$1"_log_2.txt"')

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


0

In [4]:
# Build the dictionary frame_id:timestamp
time_oracle = {}
for frame in gt:
    time_oracle[frame["frame_id"]] = frame["timestamp"]
time_oracle

{117: 47030.726,
 118: 47064.082,
 119: 47097.46,
 120: 47130.816,
 121: 47164.195,
 122: 47197.551,
 123: 47230.93,
 124: 47264.286,
 125: 47297.664,
 126: 47331.02,
 127: 47364.399,
 128: 47397.755,
 129: 47431.111,
 130: 47464.49,
 131: 47497.846,
 132: 47531.224,
 133: 47564.58,
 134: 47597.959,
 135: 47631.315,
 136: 47664.694,
 137: 47698.05,
 138: 47731.429,
 139: 47764.785,
 140: 47798.163,
 141: 47831.519,
 142: 47864.898,
 143: 47898.254,
 144: 47931.633,
 145: 47964.989,
 146: 47998.367,
 147: 48031.723,
 148: 48065.102,
 149: 48098.458,
 150: 48131.837,
 151: 48165.193,
 152: 48198.571,
 153: 48231.927,
 154: 48265.306,
 155: 48298.662,
 156: 48332.041,
 157: 48365.397,
 158: 48398.776,
 159: 48432.132,
 160: 48465.51,
 161: 48498.866,
 162: 48532.245,
 163: 48565.601,
 164: 48598.98,
 165: 48632.336,
 166: 48665.714,
 167: 48699.07,
 168: 48732.426,
 169: 48765.805,
 170: 48799.161,
 171: 48832.54,
 172: 48865.896,
 173: 48899.274,
 174: 48932.63,
 175: 48966.009,
 176: 48

In [5]:
# Merge a unique json for all frames of each camera
cameras = [6,7,8,9,10]    # 6 to 10 are the upper ones
for camera in cameras:
    cam_data = []
    directory= os.path.join(sequence_path,"trtpose3D","50_"+str(camera).zfill(2))
    for filename in sorted(os.listdir(directory)):
        with open(os.path.join(directory,filename),'r') as f:
            data  = json.load(f)
            # print(data)
            json_frame = {}
            json_frame["frame_id"] = data["frame_id"]
            
            # json_frame["timestamp"] = data["timestamp"] # This seems wrong, use the gt instead
            json_frame["timestamp"] = time_oracle[data["frame_id"]]
            
            
            json_frame["kp2d"] = []
            json_frame["kp3d"] = []
            json_frame["continuousState"] = []            
            json_frame["track_ids"] = []
            # for body in data["bodies"]:
                # pass
            if data["kp3d"][0] is not None:
                for j,body in enumerate(data["kp3d"]):
                    kp18 = np.empty((len(CONTINUOUS_STATE_PARTS),3))+np.nan
                    kp3d = {}
                    kp2d = {}
                    for i,label in enumerate(CONTINUOUS_STATE_PARTS):
                        if label in body.keys():
                            # print("2d",list(data["kp2d"][j].keys()))
                            # print("3d",list(data["kp3d"][j].keys()))
                            if '_NaN_' not in body[label]:
                                kp3d[label] = (np.array(body[label])/100).tolist()
                                # kp3d[label] = [None if x == '_NaN_' or x == 'NaN' else x/100 for x in body[label]] 
                                kp2d[label] = data["kp2d"][j][label]
                                kp18[i,:] = np.array(kp3d[label]).reshape(1,3) # cm to m

                    if kp3d != {} and kp2d != {} and len(list(kp2d.keys())) > 2: # At least 3 keypoints
                        json_frame["track_ids"].append(camera)
                        json_frame["continuousState"].append([[] if math.isnan(x[0]) else x for x in kp18.tolist()])
                        json_frame["kp3d"].append(kp3d)
                        json_frame["kp2d"].append(kp2d)
            cam_data.append(json_frame)
            # if json_frame["frame_id"] == 1169:
            #     print(os.path.join(directory,filename))
            #     break
        # dump json
    with open(os.path.join("tmp/",sequence_name+"."+ str(camera) +".json"), "w") as outfile:
        json.dump(cam_data, outfile)
    # Export in viewer format
    os.system("python3 to_viewer.py tmp/"+sequence_name+"."+ str(camera) +".json --rotation 180 --output tmp/"+sequence_name + "." + str(camera) + ".viewer.json >> tmp/$1_log_2.txt")
    # os.system("python3 to_viewer.py tmp/"+sequence_name+"."+ str(camera) +".json --rotation 180 --output tmp/"+sequence_name + "." + str(camera) + ".viewer.json")

KeyboardInterrupt: 

In [6]:
# Aggregate cameras
cameras = [6,7,8,9,10]
data = {}
for camera in cameras:
    with open(os.path.join("tmp/",sequence_name+"."+ str(camera) +".json"), "r") as f:
        file = json.load(f)
        for frame in file:
            if frame["frame_id"] not in data:
                data[frame["frame_id"]] = []
            data[frame["frame_id"]].append(frame)
        
# Make one big json file
total_out = []
for frame_id in sorted(list(data.keys())):
    total = {}
    # print(data[frame_id])
    total["frame_id"] = frame_id
    total["timestamp"] = data[frame_id][0]["timestamp"]
    total['kp2d'] = []
    total['kp3d'] = []
    total['continuousState'] = []
    total['track_ids'] = []
    for c in data[frame_id]:
        for i in range(len(c["kp2d"])):
            total["kp2d"].append(c["kp2d"][i])
            total["kp3d"].append(c["kp3d"][i])
            total["continuousState"].append(c["continuousState"][i])
            total["track_ids"].append(c["track_ids"][i])
    total_out.append(total)
    # if len(total['kp3d']) > 1:
    #     print(total)
    #     break

filename_out = sequence_name+"."+ ".".join(map(str, cameras)) +".json" 

with open(os.path.join("tmp/",filename_out), "w") as outfile:
    json.dump(total_out, outfile)
# Export in viewer format
os.system("python3 to_viewer.py tmp/"+ filename_out + " --rotation 180 --output tmp/"+ filename_out.replace('.json','.viewer.json') + " >> tmp/$1_log_2.txt")
    

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


0